In [ ]:
import numpy as np
import random

grid_size = 10

def initialize_grid(target_shape, num_obstacles=10):
    grid = np.zeros((grid_size, grid_size), dtype=int)
    
    grid[grid_size-2:grid_size, :] = 1 # bottom two rows filled with objects
    
    for _ in range(num_obstacles): # obstacles placed randomly
        x, y = random.randint(0, grid_size-3), random.randint(0, grid_size-1)
        grid[x, y] = -1  # -1 is an obstacle
    
    return grid

def get_valid_moves(grid, x, y):
    moves = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Von Neumann topology
    
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < grid_size and 0 <= ny < grid_size and grid[nx, ny] == 0:
            moves.append((nx, ny))
    
    return moves

def evaluate_grid(grid, target_shape): # heuristic function to evaluate board state
    score = 0
    for i in range(grid_size):
        for j in range(grid_size):
            if target_shape[i, j] == 1 and grid[i, j] == 1:
                score += 10  # reward correct placements
    return score

def minimax(grid, depth, alpha, beta, is_maximizing, target_shape): # Minimax algorithm with Alpha-Beta Pruning
    if depth == 0:
        return evaluate_grid(grid, target_shape)
    
    if is_maximizing:
        max_eval = -float('inf')
        for i in range(grid_size):
            for j in range(grid_size):
                if grid[i, j] == 1:
                    moves = get_valid_moves(grid, i, j)
                    for move in moves:
                        grid[i, j] = 0
                        grid[move[0], move[1]] = 1
                        eval = minimax(grid, depth-1, alpha, beta, False, target_shape)
                        grid[move[0], move[1]] = 0
                        grid[i, j] = 1
                        max_eval = max(max_eval, eval)
                        alpha = max(alpha, eval)
                        if beta <= alpha:
                            break
        return max_eval
    else:
        min_eval = float('inf')
        for i in range(grid_size):
            for j in range(grid_size):
                if grid[i, j] == 1:
                    moves = get_valid_moves(grid, i, j)
                    for move in moves:
                        grid[i, j] = 0
                        grid[move[0], move[1]] = 1
                        eval = minimax(grid, depth-1, alpha, beta, True, target_shape)
                        grid[move[0], move[1]] = 0
                        grid[i, j] = 1
                        min_eval = min(min_eval, eval)
                        beta = min(beta, eval)
                        if beta <= alpha:
                            break
        return min_eval

def find_best_move(grid, target_shape): # using minimax
    best_score = -float('inf')
    best_move = None
    
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i, j] == 1:
                moves = get_valid_moves(grid, i, j)
                for move in moves:
                    grid[i, j] = 0
                    grid[move[0], move[1]] = 1
                    score = minimax(grid, 3, -float('inf'), float('inf'), False, target_shape)
                    grid[move[0], move[1]] = 0
                    grid[i, j] = 1
                    if score > best_score:
                        best_score = score
                        best_move = (i, j, move[0], move[1])
    
    return best_move

def play_game(target_shape, auto_play=True): # runs until the shape is formed
    grid = initialize_grid(target_shape)
    while True:
        best_move = find_best_move(grid, target_shape)
        if best_move is None:
            break
        x, y, nx, ny = best_move
        grid[x, y] = 0
        grid[nx, ny] = 1
        print(grid)
        if not auto_play: # if autoplay is false then we can control manually when we advance to the next step
            input("Press Enter for the next move")

# target shape: square
target_shape = np.zeros((grid_size, grid_size), dtype=int)
target_shape[4:6, 4:6] = 1

play_game(target_shape, auto_play=True)